In [1]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image as im
from PIL.ImageShow import IPythonViewer
from scipy import ndimage
from matplotlib.pyplot import figure
from glob import glob
from tqdm import tqdm
from utils import add_margin, expand2square, binarize, put_in_a_box

from typing import *

In [23]:
def build_grid(image):
    bin_img = binarize(image)
    height, width = bin_img.shape
    
    horizontal = [0]
    encountered = False
    for idx in range(height):
        row = bin_img[idx]
        if np.sum(row) > 0:
            encountered = True
        elif encountered == True and np.sum(bin_img[idx+20, :]) == 0:
            if idx - horizontal[-1] < 20: 
                horizontal[-1] = idx
            else:
                horizontal.append(idx)
            encountered = False

    vertical = [0]
    bin_img_t = bin_img.T
    encountered = False
    for idx in range(width):
        col = bin_img_t[idx]
        if np.sum(col) > 1:
            encountered = True
        elif encountered == True and np.sum(bin_img_t[idx+20, :]) == 0:
            if idx - vertical[-1] < 20: 
                vertical[-1] = idx
            else:
                vertical.append(idx)
            encountered = False
    
    if not len(vertical) == 7 or not len(horizontal) == 12:
        print(f"{font}: vertical - {vertical}, horizontal - {horizontal}")
    assert len(vertical) == 7 # 6 vertical lines after each column + left
    assert len(horizontal) == 12 # 11 horizontal lines under each row + top
    
    return vertical, horizontal

In [24]:
pretty = put_in_a_box

In [29]:
FONTS_LOCATION = "../formatted_data/chars_images/fonts"
CHARS_LOCATION = "../formatted_data/chars_images/chars"
fonts = list(filter(lambda n: n.endswith("png"), os.listdir(FONTS_LOCATION)))

def f():
    for font in tqdm(fonts):
        img = im.open(f"{FONTS_LOCATION}/{font}")
        bin_img = binarize(img)
        vertical, horizontal = build_grid(img)

        letters = (x for x in "АБВГДЕЁЖЗИЙабвгдеёжзийКЛМНОПРСТУФклмнопрстуфХЦЧШЩЪЫЬЭЮЯхцчшщъыьэюя")
        os.makedirs(f"{CHARS_LOCATION}/{font[:-4]}", mode=0o0777, exist_ok=True)

        for left, right in [(vertical[i], vertical[i+1]) for i in range(len(vertical)-1)]:
            height, _ = bin_img.shape
            cropped = img.crop((left, 0, right, height))
            for upper, lower in [(horizontal[i], horizontal[i+1]) for i in range(len(horizontal)-1)]:
                width, _  = cropped.size
                letter = next(letters)
                char = cropped.crop((5, upper, width, lower))
                char = pretty(char)
                char.save(f"{CHARS_LOCATION}/{font[:-4]}/{letter}.png")

In [30]:
f()

100%|███████████████████████████████████████████| 17/17 [00:03<00:00,  4.30it/s]
